In [4]:
import nltk
from nltk import CFG

# 文法 

In [6]:
groucho_grammar = CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

In [11]:
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
trees = parser.parse(sent)
for tree in trees:
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


## 上下文无关文法（CFG）

In [12]:
grammar = CFG.fromstring("""
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> 'saw' | 'ate' | 'walked'
NP -> 'john' | 'Mary' | 'Bob' | Det N | Det N PP 
Det -> 'a' | 'an' | 'the' | 'my'
N -> 'man' | 'dog' | 'cat' | 'telescope' | 'park'
P -> 'in' | 'on' | 'by' | 'with'
""")

In [20]:
# sent = 'Mary saw Bob'.split()
sent = 'the dog saw a man in the park'.split()

In [21]:
rd_parser = nltk.RecursiveDescentParser(grammar)
for tree in rd_parser.parse(sent):
    print(tree)

(S
  (NP (Det the) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man) (PP (P in) (NP (Det the) (N park))))))
(S
  (NP (Det the) (N dog))
  (VP
    (V saw)
    (NP (Det a) (N man))
    (PP (P in) (NP (Det the) (N park)))))


## 使用自己的文法

In [22]:
grammar = nltk.data.load('file:mygrammar.cfg')

In [23]:
sent = 'Mary saw Bob'.split()
rd_parser = nltk.RecursiveDescentParser(grammar)
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


RecursiveDescentParser 递归下降分析器：

- 左递归产生式，如 NP -> NP PP ，会进行死循环
- 浪费了很多时间处理不符合输入句子的词和结构
- 回溯过程中可能会丢弃分析过的成分，它们将需要在之后再次重建

In [29]:
sr_parse = nltk.ShiftReduceParser(grammar)
for tree in sr_parse.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [31]:
sr_parse = nltk.ShiftReduceParser(grammar, trace=2)
for tree in sr_parse.parse(sent):
    print(tree)

Parsing 'Mary saw a dog'
    [ * Mary saw a dog]
  S [ 'Mary' * saw a dog]
  R [ NP * saw a dog]
  S [ NP 'saw' * a dog]
  R [ NP V * a dog]
  S [ NP V 'a' * dog]
  R [ NP V Det * dog]
  S [ NP V Det 'dog' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


ShiftReduceParser 移进-规约分析器：

- 不执行任何回溯，所以不能保证一定能找到一个文本的解析（即使真的存在）
- 即使有多个解析最多也只能找到一个
- 每个结构只建立一次

通过优先执行规约操作来解决移进-规约冲突

# 交互式文法编辑器
nltk.app.chartparser()

# 符合语法规则的子串表（WFST）
利用动态规划思想，结合了以上两种分析器

具体见书：270页左右

# 依存文法

In [35]:
groucho_dep_grammar = nltk.grammar.DependencyGrammar.fromstring("""
'shot' -> 'I' | 'elephant' | 'in'
'elephant' -> 'an' | 'in'
'in' -> 'pajamas'
'pajamas' -> 'my'
""")

In [36]:
print(groucho_dep_grammar)

Dependency grammar with 7 productions
  'shot' -> 'I'
  'shot' -> 'elephant'
  'shot' -> 'in'
  'elephant' -> 'an'
  'elephant' -> 'in'
  'in' -> 'pajamas'
  'pajamas' -> 'my'


In [37]:
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)
sent = 'I shot an elephant in my pajamas'.split()
trees = pdp.parse(sent)
for tree in trees:
    print(tree)

(shot I (elephant an (in (pajamas my))))
(shot I (elephant an) (in (pajamas my)))


# 特征结构

In [39]:
print(nltk.FeatStruct('''
[A = 'a',
B = (1)[C = 'c'],
D -> (1),
E -> (1)]
'''))

[ A = 'a'             ]
[                     ]
[ B = (1) [ C = 'c' ] ]
[                     ]
[ D -> (1)            ]
[ E -> (1)            ]


In [41]:
fs1 = nltk.FeatStruct(NUMBER = 74,
                     STAREET = 'run Pascal')
fs2 = nltk.FeatStruct(CITY = 'Paris')

print(fs1.unify(fs2))

[ CITY    = 'Paris'      ]
[ NUMBER  = 74           ]
[ STAREET = 'run Pascal' ]


In [42]:
fs0 = nltk.FeatStruct(A = 'a')
fs1 = nltk.FeatStruct(A = 'b')

fs2 = fs0.unify(fs1)
print(fs2)

None


In [56]:
fs1 = nltk.FeatStruct('''
[ADDRESS1 = [NUMBER = 74, STREET = 'run Pascal']]
''')
fs2 = nltk.FeatStruct('''
[ADDRESS1 = ?x,
ADDRESS2 = ?x]
''')
print(fs1)
print(fs2)

[ ADDRESS1 = [ NUMBER = 74           ] ]
[            [ STREET = 'run Pascal' ] ]
[ ADDRESS1 = ?x ]
[ ADDRESS2 = ?x ]


In [57]:
print(fs2.unify(fs1))

[ ADDRESS1 = (1) [ NUMBER = 74           ] ]
[                [ STREET = 'run Pascal' ] ]
[                                          ]
[ ADDRESS2 -> (1)                          ]
